In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from tqdm import tqdm

POSTGRES_ADDRESS = 'db.panoply.io' ## INSERT YOUR DB ADDRESS IF IT'S NOT ON PANOPLY
POSTGRES_PORT = '5439'
POSTGRES_USERNAME = 'vin.tang@gmail.com' ## CHANGE THIS TO YOURPANOPLY/POSTGRES USERNAME
POSTGRES_PASSWORD = 'Cobalt27!' ## CHANGE THIS TO YOUR PANOPLY/POSTGRES PASSWORD
POSTGRES_DBNAME = 'playerslounge_prod' ## CHANGE THIS TO YOUR DATABASE NAME

postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME,
password=POSTGRES_PASSWORD,
ipaddress=POSTGRES_ADDRESS,
port=POSTGRES_PORT,
dbname=POSTGRES_DBNAME))

cnx = create_engine(postgres_str)

In [18]:
query = """
select contests.id, contests.type, contests_plrsbefore.plr, contests.winner, contests.loser, contests_plrsbefore.userid, contests.status
from contests 
inner join contests_plrsbefore 
on contests.id = contests_plrsbefore.contests_id
where contests.console_game = 'ps4-madden21'
"""

In [19]:
df = pd.read_sql_query(query, cnx)
df.shape

(326636, 7)

In [20]:
df.head()

,id,type,plr,winner,loser,userid,status
0,7995092012829590,H,61,4.439926e+14,7.038696e+15,7038695774196162,completed
1,5179645355963421,H,54,7.893768e+15,6.613301e+15,7893767988794126,completed
2,5484846488566522,H,67,NaN,NaN,8732438145053741,cancelled
3,2230477145682123,H,84,NaN,NaN,8829685643037032,cancelled
4,3214806079020904,H,79,NaN,NaN,8245774821481175,cancelled


In [33]:
df[:500]

,id,type,plr,winner,loser,userid,status
0,7995092012829590,H,61,4.439926e+14,7.038696e+15,7038695774196162,completed
1,5179645355963421,H,54,7.893768e+15,6.613301e+15,7893767988794126,completed
2,5484846488566522,H,67,NaN,NaN,8732438145053741,cancelled
3,2230477145682123,H,84,NaN,NaN,8829685643037032,cancelled
4,3214806079020904,H,79,NaN,NaN,8245774821481175,cancelled
...,...,...,...,...,...,...,...
495,5926991958804494,H,72,5.392097e+13,2.869082e+15,2869081950979950,completed
496,6223437718902025,H,71,5.872183e+14,1.844049e+15,587218270593278,completed
497,6282502151617391,H,48,NaN,NaN,8887894137440382,cancelled
498,7370723217859565,H,63,NaN,NaN,8070627869130113,cancelled


In [41]:
lookup = {}

for _, row in tqdm(df.iterrows()):
    if row['status'] != "completed" or row['type'] != 'H':
        pass
    else:
        if row['id'] not in lookup.keys():
            if row['userid'] == row['winner']:
                lookup[row['id']] = {"winner": row['plr']}
            else:
                lookup[row['id']] = {"loser": row['plr']}
        else:
            if row['userid'] == row['winner']:
                lookup[row['id']]['winner'] = row['plr']
            else:
                lookup[row['id']]['loser'] = row['plr']

326636it [00:45, 7232.81it/s]


In [42]:
correct = 0
count = 0

for k, v in lookup.items():
    if len(v.keys()) == 2:
        count += 1
        if v['winner'] >= v['loser']:
            correct += 1

In [43]:
correct/ count

0.6235411617625489

In [45]:
lookup['5148940047099294']

{'winner': 74, 'loser': 44}